### https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [1]:
# January, 2024, High Volume For-Hire Vehicle Trip
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-01.parquet

In [2]:
!nvidia-smi

Fri Feb 21 12:19:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 6000 Ada Gene...    Off |   00000000:01:00.0 Off |                    0 |
| 30%   36C    P8             30W /  300W |   14902MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!pip freeze | grep cudf

cudf-cu12==25.2.0
cudf-polars-cu12==25.2.0
dask-cudf-cu12==25.2.0
libcudf-cu12==25.2.0
pylibcudf-cu12==25.2.0


In [4]:
!python --version

Python 3.12.3


In [5]:
pq_file = "fhvhv_tripdata_2024-01.parquet"

In [6]:
%load_ext cudf.pandas

In [7]:
import pandas as pd
display(pd)
pd.read_parquet(pq_file).head(3)

<module 'pandas' (ModuleAccelerator(fast=cudf, slow=pandas))>

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B03404,B03404,2024-01-01 00:21:47,2024-01-01 00:25:06,2024-01-01 00:28:08,2024-01-01 01:05:39,161,158,2.83,...,4.05,2.75,0.0,0.0,40.18,N,N,N,N,N
1,HV0003,B03404,B03404,2024-01-01 00:10:56,2024-01-01 00:11:08,2024-01-01 00:12:53,2024-01-01 00:20:05,137,79,1.57,...,0.89,2.75,0.0,0.0,6.12,N,N,N,N,N
2,HV0003,B03404,B03404,2024-01-01 00:20:04,2024-01-01 00:21:51,2024-01-01 00:23:05,2024-01-01 00:35:16,79,186,1.98,...,1.60,2.75,0.0,0.0,9.47,N,N,N,N,N


In [8]:
import polars as pl
pl.scan_parquet(pq_file).collect().head(3)

hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
str,str,str,datetime[ns],datetime[ns],datetime[ns],datetime[ns],i32,i32,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str
"""HV0003""","""B03404""","""B03404""",2024-01-01 00:21:47,2024-01-01 00:25:06,2024-01-01 00:28:08,2024-01-01 01:05:39,161,158,2.83,2251,45.61,0.0,1.25,4.05,2.75,0.0,0.0,40.18,"""N""","""N""","""N""","""N""","""N"""
"""HV0003""","""B03404""","""B03404""",2024-01-01 00:10:56,2024-01-01 00:11:08,2024-01-01 00:12:53,2024-01-01 00:20:05,137,79,1.57,432,10.05,0.0,0.28,0.89,2.75,0.0,0.0,6.12,"""N""","""N""","""N""","""N""","""N"""
"""HV0003""","""B03404""","""B03404""",2024-01-01 00:20:04,2024-01-01 00:21:51,2024-01-01 00:23:05,2024-01-01 00:35:16,79,186,1.98,731,18.07,0.0,0.5,1.6,2.75,0.0,0.0,9.47,"""N""","""N""","""N""","""N""","""N"""


In [9]:
pl.scan_parquet(pq_file).collect(engine="gpu").head(3)

ComputeError: AssertionError: 